In [1]:
import sys
import SimpleITK as sitk
import os
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, fixed
import math
import glob
from scipy.interpolate import RegularGridInterpolator, interpn

from skimage import measure

import pylab as pl
import trimesh
from stl import mesh
import re

from scipy.spatial import KDTree

In [2]:
#prepare data set
def centeroidnp(arr):
    """get the centroid of a point cloud"""
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return math.ceil(sum_x/length), \
            math.ceil(sum_y/length), \
            math.ceil(sum_z/length)

In [3]:
def create_7D(source_pc, source_center, target_center):
    """create a 7D pointcloud as explained in Fu et al."""
    v_s = np.zeros((len(source_pc), 7))
    for i in range(len(v_s)):
        v_ss = source_center - source_pc[i,:3]
        v_st = target_center - source_pc[i,:3]
        v_s[i,:3] = v_ss 
        v_s[i,3:6] = v_st
        v_s[i,6] = source_pc[i,3]
    return v_s


In [4]:
def set_position_and_orientation(files_deform, files_regular):
    """
    set the positioning and orientation of the deformed images so that they align
    NOTE: this should be done in the begining when the initial deformed mhd files are created
    """
    for d ,f in zip(files_deform, files_regular):
        with open(d, 'a+') as deformed:
            with open(f, 'r') as regular:
                for r in regular.readlines():
                    if "Position" in r: #or "Orientation" in r:
                        deformed.write(r)

# Make connections between vertebrae bodies and laminas with facets for XML scene in SOFA framework

In [5]:

def dist_pts(a, b):
    return np.linalg.norm(a-b)

def min_dist(points, p):
    min_=min([dist_pts(a[:3],p) for a in points])
    return [dist_pts(a[:3],p) for a in points].index(min_)

def intersect2d(X, Y):
        """
        Function to find intersection of two 2D arrays.
        Returns index of rows in X that are common to Y.
        """
        X = np.tile(X[:,:,None], (1, 1, Y.shape[0]) )
        Y = np.swapaxes(Y[:,:,None], 0, 2)
        Y = np.tile(Y, (X.shape[0], 1, 1))
        eq = np.all(np.equal(X, Y), axis = 1)
        eq = np.any(eq, axis = 1)
        return np.nonzero(eq)[0]


In [6]:
def get_bbox(position):
    """ 
    Gets the bounding box of the object defined by the given vertices.

    Arguments
    -----------
    position : list
    List with the coordinates of N points (position field of Sofa MechanicalObject).

    Returns
    ----------
    xmin, xmax, ymin, ymax, zmin, zmax : floats
    min and max coordinates of the object bounding box.
    """
    points_array = np.asarray(position)
    m = np.min(points_array, axis=0)
    xmin, ymin, zmin = m[0], m[1], m[2]

    m = np.max(points_array, axis=0)
    xmax, ymax, zmax = m[0], m[1], m[2]

    return xmin, xmax, ymin, ymax, zmin, zmax

def get_indices_in_bbox( positions, bbox ):
    """
    Get the indices of the points falling within the specified bounding box.

    Arguments
    ----------
    positions (list):
    N x 3 list of points coordinates.
    bbox (list):
    [xmin, ymin, zmin, xmax, ymax, zmax] extremes of the bounding box.

    Returns
    ----------
    indices:
    List of indices of points enclosed in the bbox.

    """
    # bbox is in the format (xmin, ymin, zmin, xmax, ...)
    assert len(bbox) == 6
    indices = []
    for i, x in enumerate( positions ):
        if x[0] >= bbox[0] and x[0] <= bbox[3] and x[1] >= bbox[1] and x[1] <= bbox[4] and x[2] >= bbox[2] and x[2] <= bbox[5]:
            indices.append( i )
    return indices
        

def print_stiff_springs(vert1,vert2, bbox_v1_v2,bbox_v2_v1, s, d):
    """
    vert1 and vert2: two adjecent vertebrae
    bbox_v1_v2 and bbox_v2_v1 are the bounding boxes representing area where the 
    springs are found on the closer sides of two adjecent vertebrae
    """
    idx1 = get_indices_in_bbox(vert1, bbox_v1_v2)[::5]
    idx2 = get_indices_in_bbox(vert2, bbox_v2_v1)[::5]
    print("SPRINGS: ")
    np.random.shuffle(idx1)
    np.random.shuffle(idx2)
    for i,j in zip(idx1,idx2):
        print("{0} {1} {2} {3} {4}  ".format(i,j,s,d,dist_pts(vert1[i],vert2[j])), end='')
        
def print_positions(vert1, bbox_v1_t12):
    """
    this function is used for seting the fixed points on L1 and L5 simulating
    connection with T12 and S1 respectively
    """
    
    idx1 = get_indices_in_bbox(vert1, bbox_v1_t12)[::5]
    print("POSITIONS:")
    for i in idx1:
        print("{0} {1} {2}  ".format(vert1[i][0],vert1[i][1],vert1[i][2]), end='')
    print("Indexes for fixed constraint")
    for i,_ in enumerate(idx1):
        print(i, end=" ")
    print("SPRINGS: ")
    for i,j in enumerate(idx1):
        print("{0} {1} {2} {3} {4}  ".format(i,j,1000,10,0.00001), end='')



## uncomment the block below to print the connections and vertices for the springs in SOFA framework AND to create biomechanical constraint files change paths and bounding boxes accordingly

In [98]:
def find_nearest_vector(array, value):
    idx = np.array([np.linalg.norm(x+y+z) for (x,y,z) in np.abs(array[:,:3]-value[:3])]).argmin()
    return int(idx)

In [99]:
def bboxes(values):
    return [np.min(values[:,0]), np.min(values[:,1]) ,np.min(values[:,2]),
      np.max(values[:,0]), np.max(values[:,1]),np.max(values[:,2])]

In [134]:
# #print bounding boxes
# for i in range(11,23):
#     print("#################################  " + str(i))
#     vert1 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v1.txt")[:,:3]
#     vert2 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v2.txt")[:,:3]
#     vert3 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v3.txt")[:,:3]
#     vert4 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v4.txt")[:,:3]
#     vert5 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v5.txt")[:,:3] 
    
#     verts = [vert1,vert2,vert3,vert4,vert5]
    
    
#     bbox_v1_t12 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v1_t12.txt")
#     bbox_v1_t12 = bboxes(bbox_v1_t12)
#     print(bbox_v1_t12)
#     bbox_v1_v2 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v2_v1.txt")
#     bbox_v1_v2 = bboxes(bbox_v1_v2)
#     print(bbox_v1_v2)
#     print(bbox_v1_v2)
#     bbox_v2_v3 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v3_v2.txt")
#     bbox_v2_v3 = bboxes(bbox_v2_v3)
#     print(bbox_v2_v3)
#     print(bbox_v2_v3)
#     bbox_v3_v4 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v4_v3.txt")
#     bbox_v3_v4 = bboxes(bbox_v3_v4)
#     print(bbox_v3_v4)
#     print(bbox_v3_v4)

#     bbox_v4_v5 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v5_v4.txt")
#     bbox_v4_v5 = bboxes(bbox_v4_v5)
#     print(bbox_v4_v5)
#     print(bbox_v4_v5)

#     bbox_v5_s1 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/v5_s1.txt")
#     bbox_v5_s1 = bboxes(bbox_v5_s1)
#     print(bbox_v5_s1)

#     values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet12.txt")
#     print(bboxes(values))
#     print(bboxes(values))

#     values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet23.txt")
#     print(bboxes(values))
#     print(bboxes(values))

#     values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet34.txt")
#     print(bboxes(values))
#     print(bboxes(values))

#     values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine"+str(i)+"/facet45.txt")
#     print(bboxes(values))
#     print(bboxes(values))
    
    
#     bboxe = [bbox_v1_t12, bbox_v1_v2,
#           bbox_v1_v2, bbox_v2_v3, 
#           bbox_v2_v3, bbox_v3_v4,
#           bbox_v3_v4, bbox_v4_v5,
#           bbox_v4_v5, bbox_v5_s1]

#     len_v = 0
#     with open("Spine"+str(i)+"_biomechanical.txt", "w+") as file:
#         indices1 = get_indices_in_bbox(verts[0], bboxe[1])
#         indices2 = get_indices_in_bbox(verts[1], bboxe[2])
#         indices3 = get_indices_in_bbox(verts[1], bboxe[3])
#         indices4 = get_indices_in_bbox(verts[2], bboxe[4])
#         indices5 = get_indices_in_bbox(verts[2], bboxe[5])
#         indices6 = get_indices_in_bbox(verts[3], bboxe[6])
#         indices7 = get_indices_in_bbox(verts[3], bboxe[7])
#         indices8 = get_indices_in_bbox(verts[4], bboxe[8])


#         a1 = find_nearest_vector(verts[0],np.mean(verts[0][indices1], axis=0))
#         a2 = find_nearest_vector(verts[1],np.mean(verts[1][indices2], axis=0))
#         a3 = find_nearest_vector(verts[1],np.mean(verts[1][indices3], axis=0))
#         a4 = find_nearest_vector(verts[2],np.mean(verts[2][indices4], axis=0))
#         a5 = find_nearest_vector(verts[2],np.mean(verts[2][indices5], axis=0))
#         a6 = find_nearest_vector(verts[3],np.mean(verts[3][indices6], axis=0))
#         a7 = find_nearest_vector(verts[3],np.mean(verts[3][indices7], axis=0))
#         a8 = find_nearest_vector(verts[4],np.mean(verts[4][indices8], axis=0))

#         file.write("{0} {1} {2} {3} {4} {5} {6} {7}".format(a1,a2,a3,a4,a5,a6,a7,a8))
#     #SANITY CHECK
# #     file.write("{0} {1} {2}\n{3} {4} {5}\n{6} {7} {8}\n{9} {10} {11}\n{12} {13} {14}\n{15} {16} {17}\n{18} {19} {20}\n{21} {22} {23}\n"
# #                                 .format(verts[0][a1][0],
# #                                         verts[0][a1][1],
# #                                         verts[0][a1][2],
# #                                         verts[1][a2][0],
# #                                         verts[1][a2][1],
# #                                         verts[1][a2][2],
# #                                          verts[1][a3][0],
# #                                         verts[1][a3][1],
# #                                         verts[1][a3][2],
# #                                         verts[2][a4][0],
# #                                         verts[2][a4][1],
# #                                         verts[2][a4][2],
# #                                          verts[2][a5][0],
# #                                         verts[2][a5][1],
# #                                         verts[2][a5][2],
# #                                         verts[3][a6][0],
# #                                         verts[3][a6][1],
# #                                         verts[3][a6][2],
# #                                          verts[3][a7][0],
# #                                         verts[3][a7][1],
# #                                         verts[3][a7][2],
# #                                         verts[4][a8][0],
# #                                         verts[4][a8][1],
# #                                         verts[4][a8][2],
# #                                                          ))

In [90]:
vert1 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v1.txt")[:,:3]
vert2 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v2.txt")[:,:3]
vert3 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v3.txt")[:,:3]
vert4 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v4.txt")[:,:3]
vert5 = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v5.txt")[:,:3] 

values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v1_t12.txt")
bbox_v1_t12 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v2_v1.txt")
bbox_v1_v2 = bboxes(values)
bbox_v2_v1 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v3_v2.txt")
bbox_v2_v3 = bboxes(values)
bbox_v3_v2 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v4_v3.txt")
bbox_v3_v4 = bboxes(values)
bbox_v4_v3 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v5_v4.txt")
bbox_v4_v5 = bboxes(values)
bbox_v5_v4 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/v5_s1.txt")
bbox_v5_s1 = bboxes(values)
bbox_v5_s1[0]+=0.012
bbox_v5_s1[3]-=0.012

values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/facet12.txt")
bbox_bone_v1_v2 = bboxes(values)
bbox_bone_v2_v1 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/facet23.txt")
bbox_bone_v2_v3 = bboxes(values)
bbox_bone_v3_v2 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/facet34.txt")
bbox_bone_v3_v4 = bboxes(values)
bbox_bone_v4_v3 = bboxes(values)
values = np.loadtxt("/Users/janelameski/Downloads/new_spines_Jane/spine22/facet45.txt")
bbox_bone_v4_v5 = bboxes(values)
bbox_bone_v5_v4 = bboxes(values)


# print_positions(vert1, bbox_v1_t12)
# print()

# print_positions(vert5, bbox_v5_s1)
# print()
# # format(i,j,500,3,dist_pts(vert1[i],vert2[j])), end='')
# print_stiff_springs(vert1, vert2, bbox_v1_v2, bbox_v2_v1,500,3)
# print()
# print_stiff_springs(vert2, vert3, bbox_v2_v3, bbox_v3_v2,500,3)
# print()
# print_stiff_springs(vert3, vert4, bbox_v3_v4, bbox_v4_v3,500,3)
# print()
# print_stiff_springs(vert4, vert5, bbox_v4_v5, bbox_v5_v4,500,3)
# # format(i,j,8000,500,dist_pts(vert1[i],vert2[j])), end='')
# print()
# print()
# print_stiff_springs(vert1, vert2, bbox_bone_v1_v2, bbox_bone_v2_v1,8000,500)
# print()
# print_stiff_springs(vert2, vert3, bbox_bone_v2_v3, bbox_bone_v3_v2,8000,500)
# print()
# print_stiff_springs(vert3, vert4, bbox_bone_v3_v4, bbox_bone_v4_v3,8000,500)
# print()
# print_stiff_springs(vert4, vert5, bbox_bone_v4_v5, bbox_bone_v5_v4,8000,500)

# vtu to txt

In [25]:
#change paths accordingly
path_vtu_files = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/"
path_txt_files = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/"
def vtu2txt(path_vtu_files, path_txt_files):
    list_files = []
    #check lines which start anything but numbers(in the vtu files the rows 
    #with numbers are the rows containing the point cloud)
    regex = re.compile("^ *<|^  +\d|^\t<|^\t\d|^\t-|^ +-")
    
    #put all vtu files in list_files
    for file in os.listdir(path_vtu_files):
        if file.endswith(".vtu"):
            list_files.append(file)
    #read all files one by one
    for file in list_files:
        with open(path_vtu_files + file, "r") as f:
            lines = f.readlines()
        #filter them using the regex above
        filtered = [i for i in lines if not regex.match(i)]
        #write them in a file with same name but ending txt
        with open(path_txt_files + file[:-3]+"txt", "w+") as f:
            for l in filtered:
                
                x,y,z = l.split(" ")
                f.write("{0} {1} {2}\n".format(float(x)*1e+3,float(y)*1e+3,float(z)*1e+3))

##  uncomment below to change vtu to point clouds txt

In [26]:
vtu2txt(path_vtu_files, path_txt_files)

## Convert vtu to obj; Uncomment to run

In [28]:
import meshio
#change path accordingly
path_to_vtu = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/"

for file in os.listdir(path_to_vtu ):
    if file.endswith(".vtu"):

        mesh_vtu = meshio.read(path_to_vtu + file)
    
        mesh = meshio.Mesh(
        mesh_vtu.points*1e3,
        mesh_vtu.cells,
        # Optionally provide extra data on points, cells, etc.
        mesh_vtu.point_data,
        # Each item in cell data must match the cells array
        mesh_vtu.cell_data,
        )
        mesh.write(path_to_vtu + "obj_files/" + file[:-3]+ "obj")


## obj to mhd is done using ImFusion

## raycast

In [106]:
def raycast_spine1(image):# spine10, spine8, spine9, spine7 , spine6 , spine 5, spine4 , spine3 , spine2 , spine 1, spine 11-19, 22
    #
    rays = np.zeros_like(np.squeeze(np.squeeze(image)))
    for i in range(image.shape[1]):
        for j in range(image.shape[0]-1, 0, -1):
            if image[j, i] != 0:
                rays[j, i] = 1
                break
    return rays

def raycast_spine8(image):# spine 8
    rays = np.zeros_like(np.squeeze(np.squeeze(image)))
    for i in range(image.shape[1]):
        for j in range(image.shape[0]):
            if image[j, i] != 0:
                rays[j, i] = 1
                break
    return rays

def raycast_spine20(image):#spine 20, 21
    rays = np.zeros_like(np.squeeze(np.squeeze(image)))
    for i in range(image.shape[0]):
        for j in range(image.shape[1]-1, 0, -1):
            if image[i, j] != 0:
                rays[i, j] = 1
                break
    return rays

def raycast_files(files_path, spine_id):
    for path in os.listdir(files_path):
        if path.endswith(".mhd") and path.startswith(spine_id):
            img_mhd = sitk.ReadImage(files_path + path)
            im = sitk.GetArrayFromImage(img_mhd)

            raycasted = np.zeros_like(im)
            
            if "e1" in spine_id or "e2_" in spine_id or "e3" in spine_id or "e4" in spine_id or "e6" in spine_id or "e7" in spine_id or "e10" in spine_id or "e22" in spine_id:
                for i in range(im.shape[0]):
                    raycasted[i,...] = raycast_spine1(im[i,...])
            elif "e5" in spine_id or "e9" in spine_id or "e8" in spine_id:
                if "e5" in spine_id or "e9" in spine_id:
                    for i in range(im.shape[2]):
                        raycasted[...,i] = raycast_spine1(im[...,i])#spine 5, 9
                elif "e8" in spine_id :
                    for i in range(im.shape[2]):
                        raycasted[...,i] = raycast_spine1(im[...,i])#spine 8,
            else:
                for i in range(im.shape[1]):
                    raycasted[:,i,:] = raycast_spine20(im[:,i,:])
            
            raycasted_img = sitk.GetImageFromArray(raycasted)
            sitk.WriteImage(raycasted_img,  files_path + "raycasted_" +path[:-3] + "mhd")
        

## Uncomment the cell below to create raycasted mhd files 

In [108]:
path_to_label_maps = "/Users/janelameski/Desktop/labelMaps/"
list_of_spine_ids = [i[:-4] for i in os.listdir(path_to_label_maps) if i.startswith("spine2") and i.endswith("mhd")]
for spine_id in list_of_spine_ids:
    raycast_files(path_to_label_maps, spine_id)
    list_raycasted = sorted([path_to_label_maps + i for i in os.listdir(path_to_label_maps) if i.startswith("raycasted_"+spine_id) and i.endswith(".mhd")])
    list_non_raycasted = sorted([path_to_label_maps + i for i in os.listdir(path_to_label_maps) if i.startswith(spine_id) and i.endswith(".mhd")])
    set_position_and_orientation(list_raycasted, list_non_raycasted)
    


## mhd to .txt is done using ImFusion

# Prepare spine data .txt to .npz

In [121]:
list_files = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/" + "txtFiles/"
def obtain_file_id(p):
    """
    IMPORTANT: make sure that the path to files folder (p) does not contain "_" except in the name of the 
    files themselves
    """
    splits = p.split("_")
    #in case the label of the spine is double digit
    if splits[0][-2:].isnumeric():
        #dealing with Source point clouds "spine11_vert10.txt"
        if len(splits)==2:
            s_id = splits[0][-7:] + splits[1][5]
        #this is the case only for several target files such as spine11_vert2_1.txt
        elif len(splits)==3 and splits[2].endswith(".txt"):
            s_id = splits[0][-7:] + "_" +splits[2][0]    
        #for all other target files such as "spine11_vert5_1_0"
        else:
            s_id = splits[0][-7:] + "_"+splits[2]+"_"+splits[3][0]
    else:  
         #dealing with Source point clouds "spine1_vert10.txt"
        if len(splits)==2:
            s_id = splits[0][-6:] + splits[1][5]
        #this is the case only for several target files such as spine1_vert2_1.txt
        elif len(splits)==3 and splits[2].endswith(".txt"):
            s_id = splits[0][-6:] + "_" +splits[2][0]    
        #for all other target files such as "spine1_vert5_1_0"
        else:
            s_id = splits[0][-6:] + "_"+splits[2]+"_"+splits[3][0]
    return s_id

def obtain_indices_raycasted_original_pc(spine_target, r_target):
    """
    find indices in spine_target w.r.t. r_target such that they are the closest points between the two 
    point clouds
    """
    kdtree=KDTree(spine_target[:,:3])
    dist,points=kdtree.query(r_target[:,:3],1)

    return list(set(points))

def create_source_target_with_vertebra_label(source_pc, target_pc, vert):
    """
    source_pc: source point cloud
    target_pc: target point cloud
    vert: [1-5] for [L1-L5] vertebra respectively
    
    this function is to create source and target point clouds with label for each vertebra
    """
    
    source = np.ones((source_pc.shape[0], source_pc.shape[1]+1))
    source[:, :3]=source_pc
    source[:, 3] = source[:, 3]*vert
    target = np.ones((target_pc.shape[0], target_pc.shape[1]+1))
    target[:, :3]=target_pc
    target[:, 3]= target[:, 3]*vert
    
    return source, target

def create_source_target_flow_spine(source_pc, target_pc):
    """
    source_pc: source point cloud
    target_pc: target point cloud
    vert: [1-5] for [L1-L5] vertebra respectively
    
    this function is to create source and target point clouds with 7D
    where the point clouds are centered.
    """
    
#     source_pc, target_pc = create_source_target_with_vertebra_label(source_pc, target_pc, vert)

    centroid_source = centeroidnp(source_pc)
    centroid_target = centeroidnp(target_pc)
    
    source_7d = create_7D(source_pc, centroid_source, centroid_target)
    target_7d = create_7D(target_pc, centroid_source, centroid_target)
    
    flow = target_7d[:,:3]-source_7d[:,:3]
    
    return source_7d, target_7d, flow



def write_source_target_flow_as_npz(pa, num):
    """
    USE THIS METHOD TO CREATE FULL SPINE DATA AS .npz
    
    pa: path to vertebrae data set with name of files: 
    "spine{num_spine}_vert{num_vert}_{SOFA_experiment}_{Nx20_iterations}.txt"
    num: which patient does the spine belong to labeled from 1-10
    
    returns a list of spine point clouds containing flow source target, 
    and another list with the constraints
    """
    source_rgx = re.compile("^spine"+str(num)+"_vert.0")
    l1_rgx = re.compile("^spine"+str(num)+"_vert1_")
    l2_rgx = re.compile("^spine"+str(num)+"_vert2_")
    l3_rgx = re.compile("^spine"+str(num)+"_vert3_")
    l4_rgx = re.compile("^spine"+str(num)+"_vert4_")
    l5_rgx = re.compile("^spine"+str(num)+"_vert5_")
    
    source_list = []
    #if the path to the biomechanical constraints is different, add it accordingly 
    constraint_list = np.loadtxt("Spine"+str(num)+"_biomechanical.txt")
    l1=[]
    l2=[]
    l3=[]
    l4=[]
    l5=[]
    
    #list the files matching the regex above to their name
    for file in os.listdir(pa):
        if file.endswith(".txt") and source_rgx.match(file):
            source_list.append(os.path.join(pa, file))
        if file.endswith(".txt") and l1_rgx.match(file):
            l1.append(os.path.join(pa, file))
        if file.endswith(".txt") and l2_rgx.match(file):
            l2.append(os.path.join(pa, file))
        if file.endswith(".txt") and l3_rgx.match(file):
            l3.append(os.path.join(pa, file))
        if file.endswith(".txt") and l4_rgx.match(file):
            l4.append(os.path.join(pa, file))
        if file.endswith(".txt") and l5_rgx.match(file):
            l5.append(os.path.join(pa, file))
    
    source_list = sorted(source_list)
    targets = (sorted(l1),sorted(l2),sorted(l3),sorted(l4),sorted(l5))

    list_of_triples = []
    list_of_spines = []
    for i, t in enumerate(targets):
        
        for p in t:
            sour, tar = create_source_target_with_vertebra_label(np.loadtxt(source_list[i]), np.loadtxt(p), i+1)
            t_id = obtain_file_id(p)
            s_id = obtain_file_id(source_list[i])
            list_of_triples.append([sour, tar, t_id])

        list_of_spines.append(list_of_triples)
        list_of_triples=[]

    #for every vertebra in the combined files
    for t1,t2,t3,t4,t5 in zip(list_of_spines[0], list_of_spines[1], list_of_spines[2],
                         list_of_spines[3], list_of_spines[4]):
        
        #create the whole spines (source and target)
        spine_source = np.concatenate((t1[0],t2[0],t3[0],t4[0],t5[0]))
        spine_target = np.concatenate((t1[1],t2[1],t3[1],t4[1],t5[1]))
        
        source, target, flow = create_source_target_flow_spine(spine_source, spine_target)
    
        target_id = t1[2]
        
        np.savez_compressed(pa + target_id + ".npz",
                                    flow=flow,
                                    pc1=source,
                                    pc2=target,
                                    cstPts=constraint_list)


def combine_vertebrae_with_label(pa, num, raycasted_path):
    """
    USE THIS METHOD TO CREATE RAY CASTED SPINE DATA AS .npz 
    
    pa: path to vertebrae data set with name of files:
    "spine{num_spine}_vert{num_vert}_{SOFA_experiment}_{Nx20_iterations}.txt"
    num: which spine we are transforming
    raycasted_path: path where the raycasts are saved as
    """
    
    #select the file names according to whether they are source "spine1_vert10.txt" or target "spine1_vert1_1_0"
    source_rgx = re.compile("^spine"+str(num)+"_vert.0")
    l1_rgx = re.compile("^spine"+str(num)+"_vert1_")
    l2_rgx = re.compile("^spine"+str(num)+"_vert2_")
    l3_rgx = re.compile("^spine"+str(num)+"_vert3_")
    l4_rgx = re.compile("^spine"+str(num)+"_vert4_")
    l5_rgx = re.compile("^spine"+str(num)+"_vert5_")
    
    l1=[]
    l2=[]
    l3=[]
    l4=[]
    l5=[]
    
    source_list = []
    
    #list the files matching the regex above to their name
    for file in os.listdir(pa):
        if file.endswith(".txt") and source_rgx.match(file):
            source_list.append(os.path.join(pa, file))
        if file.endswith(".txt") and l1_rgx.match(file):
            l1.append(os.path.join(pa, file))
        if file.endswith(".txt") and l2_rgx.match(file):
            l2.append(os.path.join(pa, file))
        if file.endswith(".txt") and l3_rgx.match(file):
            l3.append(os.path.join(pa, file))
        if file.endswith(".txt") and l4_rgx.match(file):
            l4.append(os.path.join(pa, file))
        if file.endswith(".txt") and l5_rgx.match(file):
            l5.append(os.path.join(pa, file))
            
    source_list = sorted(source_list)
    targets = (sorted(l1),sorted(l2),sorted(l3),sorted(l4),sorted(l5))

    list_of_triplets = []
    list_of_spines = []
    

    #combine the files in a list of lists containing [source, target, source_id, target_id]
    for i, t in enumerate(targets):
        for p in t:
            sour, tar = create_source_target_with_vertebra_label(np.loadtxt(source_list[i]), np.loadtxt(p), i+1)
            t_id = obtain_file_id(p)
            s_id = obtain_file_id(source_list[i])
            list_of_triplets.append([sour,tar,s_id, t_id])
        list_of_spines.append(list_of_triplets)
        list_of_triplets = []
    spine_source = []
    spine_target = []
    
    #load the biomechanical constraint point indices
    bio_constraints = np.loadtxt("Spine"+str(num)+"_biomechanical.txt")
    
    #for every vertebra in the combined files
    for t1,t2,t3,t4,t5 in zip(list_of_spines[0], list_of_spines[1], list_of_spines[2],
                         list_of_spines[3], list_of_spines[4]):
        
        #create the whole spines (source and target)
        spine_source = np.concatenate((t1[0],t2[0],t3[0],t4[0],t5[0]))
        spine_target = np.concatenate((t1[1],t2[1],t3[1],t4[1],t5[1]))
        
        #create the flow
        spine_flow = spine_target[:,:3] - spine_source[:,:3]
        
        #the id of the spine
        source_id = t1[2]
        target_id = t1[3]
        
        #find the respective points for the biomechanical constraint point indices
        bio_constraints = [int(i) for i in bio_constraints]
        bio_points = [find_nearest_vector(spine_source, t1[0][bio_constraints[0]]),
                      find_nearest_vector(spine_source, t2[0][bio_constraints[1]]),
                      find_nearest_vector(spine_source, t2[0][bio_constraints[2]]),
                      find_nearest_vector(spine_source, t3[0][bio_constraints[3]]),
                      find_nearest_vector(spine_source, t3[0][bio_constraints[4]]),
                      find_nearest_vector(spine_source, t4[0][bio_constraints[5]]),
                      find_nearest_vector(spine_source, t4[0][bio_constraints[6]]),
                      find_nearest_vector(spine_source, t5[0][bio_constraints[7]])]
        

        #load the extracted point clouds from the .mhd ray casts
        r_source = np.loadtxt(raycasted_path + "raycasted_" + source_id + ".txt")
        r_target = np.loadtxt(raycasted_path + "raycasted_" + target_id + ".txt")
        
        raycasted_source_idcs = obtain_indices_raycasted_original_pc(spine_source, r_source)
        raycasted_target_idcs = obtain_indices_raycasted_original_pc(spine_target, r_target)
        #add the biomechanical constraint points in the raycasted_source because they are probably
        #not part of the point cloud
        raycasted_source_idcs.extend(bio_points)
        
        #select only the ray casted points from the original source and target
        raycasted_source = spine_source[raycasted_source_idcs, :]
        raycasted_target = spine_target[raycasted_target_idcs, :]
        raycasted_source_flow = np.zeros_like(raycasted_source)
        raycasted_source_flow[:,:3] = raycasted_source[:,:3] + spine_flow[raycasted_source_idcs, :]
        raycasted_source_flow[:,3] = raycasted_source[:,3]
        
        #prepare data into 7D 
        centroid_source = centeroidnp(raycasted_source)
        centroid_target = centeroidnp(raycasted_target)
        centroid_source_flow = centeroidnp(raycasted_source_flow)
        
        source_7d = create_7D(raycasted_source, centroid_source, centroid_source_flow)
        target_7d = create_7D(raycasted_target, centroid_source, centroid_target)
        flow_source_7d = create_7D(raycasted_source_flow, centroid_source, centroid_source_flow)

        flow = flow_source_7d[:,:3]-source_7d[:,:3]
        
    
        #find the biomechanical points in every vertebra 
        surface1 = np.copy(raycasted_source[:,3])
        L1 = raycasted_source[np.argwhere(surface1 == 1.).squeeze()]
        L2 = raycasted_source[np.argwhere(surface1 == 2.).squeeze()]
        L3 = raycasted_source[np.argwhere(surface1 == 3.).squeeze()]
        L4 = raycasted_source[np.argwhere(surface1 == 4.).squeeze()]
        L5 = raycasted_source[np.argwhere(surface1 == 5.).squeeze()]
        
        #create the .npz file
        np.savez_compressed(pa + "raycasted_"+ target_id + ".npz",
                                    flow=flow,
                                    pc1=source_7d,
                                    pc2=target_7d,
                                    cstPts=np.array([L1.shape[0]-1,
                                                    L2.shape[0]-1,L2.shape[0]-2,
                                                    L3.shape[0]-1,L3.shape[0]-2,
                                                   L4.shape[0]-1,L4.shape[0]-2,
                                                   L5.shape[0]-1]))


## uncomment the cell below and change paths accordingly to create files from full spine data

In [132]:
# path_to_npz = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/" + "txtFiles/"

# for i in range(22):
#     write_source_target_flow_as_npz(path_to_npz, i+1)

## Uncomment the cell below and change paths accordingly to create npz training files from raycasted data

In [133]:
# path_to_npz = "/Users/janelameski/Desktop/jane/sofa/SOFAZIPPED/install/bin/" + "txtFiles/"
# path_to_label_maps = "/Users/janelameski/Desktop/labelMaps/"

# for i in range(22):
#     combine_vertebrae_with_label(path_to_npz, i+1, path_to_label_maps)